In [ ]:
import os
import sys


def setup_project_root(start_path='.'):
    """Find the project root, set it as the current working directory, and add it to sys.path."""
    current_path = os.path.abspath(start_path)
    while True:
        if '.git' in os.listdir(current_path):
            project_root = current_path
            break
        parent_path = os.path.dirname(current_path)
        if parent_path == current_path:  # We've reached the root directory
            raise Exception("Could not find project root (.git directory not found)")
        current_path = parent_path
    
    # Change the current working directory to the project root
    os.chdir(project_root)
    print(f"Current working directory set to: {os.getcwd()}")

    # Add project root to sys.path if it's not already there
    if project_root not in sys.path:
        sys.path.insert(0, project_root)
        print(f"Added {project_root} to sys.path")

# sets the current working directory to the project root
setup_project_root()

# Don't cache imports
%load_ext autoreload
%autoreload 2


In [ ]:
from info_theory_experiments.custom_datasets import FMRIDatasetConcat
import torch
from torch import nn, optim
from torch.utils.data import DataLoader

class Autoencoder(nn.Module):
    def __init__(self, input_size, feature_size, hidden_sizes):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_sizes[0]),
            nn.ReLU(True),
            nn.Linear(hidden_sizes[0], hidden_sizes[1]),
            nn.ReLU(True),
            nn.Linear(hidden_sizes[1], hidden_sizes[2]),
            nn.ReLU(True),
            nn.Linear(hidden_sizes[2], hidden_sizes[3]),
            nn.ReLU(True),
            nn.Linear(hidden_sizes[3], feature_size)
        )
        self.decoder = nn.Sequential(
            nn.Linear(feature_size, hidden_sizes[3]),
            nn.ReLU(True),
            nn.Linear(hidden_sizes[3], hidden_sizes[2]),
            nn.ReLU(True),
            nn.Linear(hidden_sizes[2], hidden_sizes[1]),
            nn.ReLU(True),
            nn.Linear(hidden_sizes[1], hidden_sizes[0]),
            nn.ReLU(True),
            nn.Linear(hidden_sizes[0], input_size)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

seed = 42 
torch.manual_seed(seed)


In [ ]:

dataset = FMRIDatasetConcat()
device = 'cuda'

for i in range(1):
    config = {
        "torch_seed": seed,
        "dataset_type": "FMRI",
        "nuem_atoms": 100,
        "batch_size": 1000,
        "train_mode": True,
        "train_model_B": False,
        "adjust_Psi": False,
        "clip": 5,
        "feature_size": 3,
        "epochs": 75,
        "hidden_sizes": [256, 256, 256, 256, 256],
        "lr": 1e-4,
        "weight_decay": 1e-6,
    }

    torch.manual_seed(config['torch_seed'])
    trainloader = DataLoader(dataset, batch_size=config['batch_size'], shuffle=True)

    input_size = 100
    model = Autoencoder(input_size, config['feature_size'], config['hidden_sizes']).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])

    for epoch in range(config['epochs']):
        for data in trainloader:
            data = data.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, data)
            loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch+1}/{config["epochs"]}], Loss: {loss.item():.4f}')

    torch.save(model.state_dict(), 'models/FMRI-data-autoencoder_model.pth')



# In this experiment we check if a representation learned by an autoencoder rep is emergent

In [ ]:
seed = 42
def get_autoencoder_representation(batch_data):
    """
    Function to get the autoencoder representation of a batch of data.

    Parameters:
    - batch_data: Tensor containing the batch of data.

    Returns:
    - encoded_data: Tensor containing the autoencoder representations.
    """
    # Load the model
    model = Autoencoder(input_size, config['feature_size'], config['hidden_sizes']).to(device)
    model.load_state_dict(torch.load('models/FMRI-data-autoencoder_model.pth'))
    model.eval()  # Set the model to evaluation mode

    with torch.no_grad():  # Disable gradient calculation
        batch_data = batch_data.to(device)
        encoded_data = model.encoder(batch_data)

    return encoded_data



project_name = "NEURIPS-testing-if-encoder-rep-is-emergent"

config_test = {    
    "torch_seed": seed,
    "dataset_type": "FMRI",
    "num_atoms": 100,
    "batch_size": 1000,
    "train_mode": False,
    "train_model_B": False,
    "adjust_Psi": True,
    "clip": 5,
    "feature_size": 3,
    "epochs": 10,
    "start_updating_f_after": 100,
    "update_f_every_N_steps": 5,
    "minimize_neg_terms_until": 0,
    "downward_critics_config": {
        "hidden_sizes_v_critic": [512, 1024, 1024, 512],
        "hidden_sizes_xi_critic": [512, 512, 512],
        "critic_output_size": 32,
        "lr": 1e-3,
        "bias": True,
        "weight_decay": 0,
    },
    
    "decoupled_critic_config": {
        "hidden_sizes_encoder_1": [512, 512, 512],
        "hidden_sizes_encoder_2": [512, 512, 512],
        "critic_output_size": 32,
        "lr": 1e-3,
        "bias": True,
        "weight_decay": 0,
    },
    "feature_network_config": {
        "hidden_sizes": [256, 256, 256, 256, 256],
        "lr": 1e-4,
        "bias": True,
        "weight_decay": 1e-6,
    }
}

from info_theory_experiments.trainer_for_testing_function_psi import train_feature_network

out = train_feature_network(
    config=config_test,
    trainloader=trainloader,
    feature_network_training=get_autoencoder_representation,
    project_name=project_name,
)

